In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/09/26 20:27:35 WARN Utils: Your hostname, MAGI resolves to a loopback address: 127.0.1.1; using 172.22.235.175 instead (on interface eth0)
23/09/26 20:27:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/26 20:27:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/26 20:27:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [4]:
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")], ["label", "sentence"])

In [5]:
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish Java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [6]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [7]:
words_data = tokenizer.transform(sentenceData)

In [8]:
words_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+



In [9]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [10]:
featurized_data = hashing_tf.transform(words_data)

In [11]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [12]:
idf_model = idf.fit(featurized_data)

In [13]:
rescaled_data = idf_model.transform(featurized_data)

In [14]:
rescaled_data.select('label', 'features').show()

23/09/26 20:27:46 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
23/09/26 20:27:46 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262144,[18700,19...|
|  0.0|(262144,[19036,20...|
|  1.0|(262144,[46243,58...|
+-----+--------------------+



23/09/26 20:27:46 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


In [15]:
from pyspark.ml.feature import CountVectorizer

In [16]:
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))], ["id", "words"])

In [17]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [18]:
cv = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF=2.0)

In [19]:
model = cv.fit(df)

In [20]:
result = model.transform(df)

In [21]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

